# Tutoriel MID Contrôle
<u>Objectif</u> : Illustrer les propriétés de la MID sous la forme contrôle.

Concrètement, nous considérons un système qui a un controle par une loi de commande retardée (expression mathématique) et l'on prend compte et tirons avantage de ce retard dans le calcul des gains de la loi de commande retardée pour stabiliser le système.

<u> Informations importantes </u> :

- Stabilité : Lorsque l'on a l'équation décrivant un système dans l'espace de Laplace (transformée de Laplace), la stabilité est traduite par l'ensemble des racines (complexes) de l'équation ayant leurs parties réelles négatives. On dit que les racines sont à gauche (de l'axe des ordonnées).
- Transformée de Laplace (juste pour info) : Dans notre cas, la partie la plus importante dans la transformée de Laplace est la dérivation, c'est ce qui nous permet de passer d'une équation différentielle à une équation polynomiale. L'idée très grossière est qu'une dérivée d'ordre $n$ dans l'espace "classique"/"temporel" correspond à une multiplication par un terme $s^n$ dans l'espace de Laplace (fréquentiel). Donc, par exemple, $4\times k \times y''(t)\rightarrow 4ks^2y(s)$

On traite une équation différentielle du type :
$$
\frac{d^n y(t)}{dt^n} + \sum\limits_{i=0}^{n-1}a_i\frac{d^i y(t)}{dt^i} + \sum\limits_{j=0}^m \alpha_j\frac{d^j y(t-\tau)}{dt^j}=0
$$

- $n, m\in\mathbb{N}, n>m$
- $a_i, \alpha_j \in \mathbb{R}$
- $s\in\mathbb{C}$,
- $\tau \in \mathbb{R}$

Par une transformée de Laplace :
$$
Q(s) = s^n+a_{n-1}s^{n-1}+...+a_0 + \left[\alpha_ms^m+...+\alpha_0\right]e^{-s\tau}
$$

Cette équation est polynomiale et non différentielle ce qui la rend beaucoup plus simple à traiter.

Cette fois-ci, on considère que l'on connait les paramètres du systèmes $a_i$ ainsi que l'un des hyperparamètres $s_0$ ou $\tau$. Ainsi, nous allons utiliser la MID pour trouver les $\alpha_j$ et l'hyperparamètre manquant.

Le code suivant a été développé sur la base du pseudo-code de la MID Générique, décrit par Franck, puis adapté en version interactive.

Version : 1.1 (22/04/2021) - test pull



In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 44em; }</style>"))
%matplotlib notebook
import numpy as np
import sympy as sp
import cxroots as cx
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import matplotlib.cm as cm

def compute_admissibilite(n,m,value_s0,value_tau):
    
    s = sp.symbols('s')  # define variable s for our problem to be solved
    tau = sp.symbols('tau')  # define variable tau : delay

    a = sp.symbols(["a{:d}".format(i) for i in range(n)], real=True)
    alpha = sp.symbols(["alpha{:d}".format(i) for i in range(m + 1)], real=True)
    avalue = [1, 1]

    Polynomial = s**n + np.array(a).dot([s**i for i in range(n)]) # Revient à faire s^n + a_{n-1}^{n-1}...
    Delayed = np.array(alpha).dot([s**i for i in range(m+1)])*sp.exp(-s*tau) # Revient à faire 
    #b^m*s^m + b_{m-1}^{m-1}...
    Q = Polynomial + Delayed 

    SysDerivatif = [Q]
    for i in range(m+1):
        DerniereDerivee = SysDerivatif[-1]
        SysDerivatif.append(DerniereDerivee.diff(s)) # Dérivée par rapport à s

    sol = sp.linsolve(SysDerivatif[:-1], alpha).args[0] # Solveur selon les alpha et les a

    finaleq = SysDerivatif[-1].subs({alph : alphacoef for alph, alphacoef in zip(alpha, sol)}) #remplace les coeffs
    finaleq = finaleq.subs({asymb: aval for asymb, aval in zip(a, avalue)})
    solS0 = finaleq.subs({tau : value_tau})
    solS0 = sp.solve(solS0)
    solS0eval = [i.evalf() for i in solS0]

    computedS0 = solS0[1]
    alpha_num = sol.subs({asymb: aval for asymb, aval in zip(a, avalue)})
    alpha_num = alpha_num.subs({s : computedS0})
    alpha_num = alpha_num.subs({tau : value_tau})
    alpha_num_eval = [i.evalf() for i in alpha_num]
    alpha_sens = alpha_num_eval

    finaleq = SysDerivatif[-1].subs({alph : alphacoef for alph, alphacoef in zip(alpha, sol)}) #remplace les coeffs
    finaleq = finaleq.subs({asymb: aval for asymb, aval in zip(a, avalue)})
    solTau = finaleq.subs({s : value_s0})
    solTau = sp.solve(solTau)

    computedTau = solTau[0]
    alpha_num = sol.subs({asymb: aval for asymb, aval in zip(a, avalue)})
    alpha_num = alpha_num.subs({tau : computedTau})
    alpha_num = alpha_num.subs({s : value_s0})
    alpha_num_eval = [i.evalf() for i in alpha_num]

    polyAdm = SysDerivatif[-1].subs({alph : alphacoef for alph, alphacoef in zip(alpha, sol)})
    polyAdm = polyAdm.subs({asymb: aval for asymb, aval in zip(a, avalue)})
    polyAdm = sp.simplify(polyAdm)

    s0range = np.arange(-10, 0, 0.01)
    taurange = np.arange(0, 10, 0.01)

    func = sp.lambdify([s, tau], polyAdm)

    return s0range,taurange,polyAdm,s,tau,Q,avalue,alpha_sens,SysDerivatif,alpha,a

def getRoots(m, Q, dominancy, delay, avalue, alphavalue, xwindow, ywindow,s,a,alpha,tau):
    derivees = [Q, Q.diff(s)]
    for i in range(len(derivees)) :
        derivees[i] = derivees[i].subs({ai: ai_num for ai, ai_num in zip(a, avalue)})
        derivees[i] = derivees[i].subs({alphai: alphai_num for alphai, alphai_num in zip(alpha, alphavalue)})
        derivees[i] = derivees[i].subs({tau : delay})
    func = [sp.lambdify(s, i) for i in derivees]
    rect = cx.Rectangle(xwindow, ywindow)
    root_count = rect.count_roots(func[0])
    roots = rect.roots(func[0], func[1], rootErrTol=1e-5, absTol=1e-5, M=m+2)
    xroot, yroot = np.real(roots[0]), np.imag(roots[0])
    return xroot, yroot

def solve_tau_connu(tau_val, acoef, Q,m,s,SysDerivatif,alpha,a,tau):
    sys = [Q]
    for i in range(m+1):
        DerniereDerivee = sys[-1]
        sys.append(DerniereDerivee.diff(s)) # Dérivée par rapport à s
    sol = sp.linsolve(SysDerivatif[:-1], alpha).args[0]
    finaleq = SysDerivatif[-1].subs({alph : alphacoef for alph, alphacoef in zip(alpha, sol)}) 
    finaleq = finaleq.subs({asymb: aval for asymb, aval in zip(a, acoef)})
    solS0 = finaleq.subs({tau : tau_val})
    solS0 = sp.solve(solS0)
    solS0eval = [i.evalf() for i in solS0]
    try :
        solution = max([i for i in solS0eval if i<0])
    except Exception :
        traceback.print_exc()
    return solution

def compute_sensibilite(value_tau,Q,m,s,SysDerivatif,alpha,a,tau):
    tau_nominal = value_tau
    step = 1e-2
    nbIt = 10
    values = [0] + [-step * i for i in range(1, nbIt + 1)] + [step * i for i in range(1, nbIt + 1)]
    values.sort()
    tau_sens = []
    s0_sens = []
    for value in values :
        tau_sens.append(tau_nominal + value)
        s0_sens.append(solve_tau_connu(tau_sens[-1], [2, 1], Q,m,s,SysDerivatif,alpha,a,tau))
        
    sensIterations = len(tau_sens)
    bList = np.linspace(start=255, stop=0, num=sensIterations//2) #Blue to black
    rList = np.linspace(start=0, stop=255, num=sensIterations//2) #black to red
    if len(bList)+len(rList)==sensIterations-1 :
        zerosToFill = [0] * (sensIterations//2 + 1)
    else :
        zerosToFill = [0] * (sensIterations//2 )
    bList = [*bList, *zerosToFill]
    rList = [*zerosToFill, *rList]
    colorArray = []
    for i in range(len(tau_sens)):
        colorArray.append([rList[i], 0, bList[i]])
    colorArray = np.array(colorArray)
    
    return tau_sens,s0_sens,colorArray

In [2]:
output_admissibilite = widgets.Output()
output_equation = widgets.Output()
output_sensibilite = widgets.Output()

slider_n = widgets.IntSlider(min=0,max=2,step=1,description='n :',value=2)
slider_m = widgets.IntSlider(min=0,max=3,step=1,description='m :',value=1)
FloatText_s0 = widgets.BoundedFloatText(value=-1.5,min=-2.0,max=0.0,step=0.1,description='s0:',disabled=False)
FloatText_tau = widgets.BoundedFloatText(value=0.73,min=0.50,max=1.00,step=0.01,description='tau:',disabled=False)

      
def common_slider(n,m,value_s0,value_tau):
    output_admissibilite.clear_output()
    output_equation.clear_output()
    output_sensibilite.clear_output()
    
    with output_admissibilite:
        
        # additional calculations for the plot 
        s0range,taurange,polyAdm,s,tau,Q,avalue,alpha_sens,SysDerivatif,alpha,a = compute_admissibilite(n,m,value_s0,value_tau)

        s0range = np.arange(-10, 0, 0.01)
        taurange = np.arange(0, 10, 0.01)

        func = sp.lambdify([s, tau], polyAdm)
    
        fig, ax = plt.subplots()
        X, Y = np.meshgrid(s0range, taurange)
        z = func(X, Y)
        CS = ax.contour(X, Y, z, [0])
        ax.grid()
        plt.xlabel(r"$s_0$")
        plt.ylabel(r"$\tau$")
        plt.title("Admissibility plot")
    
        
    with output_equation :
        print ("Eq = ",polyAdm)
    
    with output_sensibilite :
        
        # additional calculations for the plot
        tau_sens,s0_sens,colorArray = compute_sensibilite(value_tau,Q,m,s,SysDerivatif,alpha,a,tau)
        
        scaler = lambda x : [i / 255 for i in x]
        
        plt.figure()
        for i in range(len(tau_sens)):
            xroot, yroot = getRoots(1, Q, s0_sens[i], tau_sens[i],
                                    avalue, alpha_sens, [-5, 5], [-5, 5],s,a,alpha,tau)
            plt.scatter(xroot, yroot, c=[scaler(colorArray[i])])
            
        plt.grid()
        plt.xlabel(r"$Re(s)$")
        plt.ylabel(r"$Im(s)$")
        tau_sens.sort()
        plt.title(r"Sensitivity plot, $\tau \in $"+f"[{tau_sens[0]}, {tau_sens[-1]}]")
        

def slider_n_eventhandler(change):
    common_slider(change.new, slider_m.value, FloatText_s0.value, FloatText_tau.value)
def slider_m_eventhandler(change):
    common_slider(slider_n.value, change.new, FloatText_s0.value, FloatText_tau.value)
def FloatText_s0_eventhandler(change):
    common_slider(slider_n.value, slider_m.value, change.new, FloatText_tau.value)
def FloatText_tau_eventhandler(change):
    common_slider(slider_n.value, slider_m.value, FloatText_s0.value, change.new)


slider_n.observe(slider_n_eventhandler, names='value')
slider_m.observe(slider_m_eventhandler, names='value')
FloatText_s0.observe(FloatText_s0_eventhandler, names='value')
FloatText_tau.observe(FloatText_tau_eventhandler, names='value')

input_widgets = widgets.HBox([slider_n, slider_m, FloatText_s0, FloatText_tau])

tab = widgets.Tab([output_admissibilite, output_equation, output_sensibilite])
tab.set_title(0, 'Plot Admissibility ')
tab.set_title(1, 'Output equation')
tab.set_title(2, 'Plot sensitivity')

dashboard = widgets.VBox([widgets.Label('Insert degree of polynomial n, degree of delay polynomial m, s0 and tau :'),input_widgets, tab])

In [3]:
display(dashboard)